### N-gram language models or how to write scientific papers (4 pts)

We shall train our language model on a corpora of [ArXiv](http://arxiv.org/) articles and see if we can generate a new one!

![img](https://media.npr.org/assets/img/2013/12/10/istock-18586699-monkey-computer_brick-16e5064d3378a14e0e4c2da08857efe03c04695e-s800-c85.jpg)

_data by neelshah18 from [here](https://www.kaggle.com/neelshah18/arxivdataset/)_

_Disclaimer: this has nothing to do with actual science. But it's fun, so who cares?!_

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Alternative manual download link: https://yadi.sk/d/_nGyU2IajjR9-w
!wget "https://www.dropbox.com/s/99az9n1b57qkd9j/arxivData.json.tar.gz?dl=1" -O arxivData.json.tar.gz
!tar -xvzf arxivData.json.tar.gz
data = pd.read_json("./arxivData.json")

In [ ]:
data.sample(n=3)

,author,day,id,link,month,summary,tag,title,year
6380,[{'name': 'Kevin K. Chen'}],29,1611.09448v2,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",11,Neural networks with rectified linear unit act...,"[{'term': 'stat.ML', 'scheme': 'http://arxiv.o...",The Upper Bound on Knots in Neural Networks,2016
26703,"[{'name': 'A. P. Prathosh'}, {'name': 'Pragath...",21,1611.06674v1,"[{'rel': 'related', 'href': 'http://dx.doi.org...",11,Non-contact estimation of respiratory pattern ...,"[{'term': 'cs.CV', 'scheme': 'http://arxiv.org...",Estimation of respiratory pattern from video u...,2016
21896,[{'name': 'Tianwen Wei'}],28,1408.6693v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",8,The FastICA algorithm is one of the most popul...,"[{'term': 'stat.ML', 'scheme': 'http://arxiv.o...",A study of the fixed points and spurious solut...,2014


In [3]:
# assemble lines: concatenate title and description
lines = data.apply(lambda row: row['title'] + ' ; ' + row['summary'].replace("\n", ' '), axis=1).tolist()

sorted(lines, key=len)[:3]

['Differential Contrastive Divergence ; This paper has been retracted.',
 'What Does Artificial Life Tell Us About Death? ; Short philosophical essay',
 'P=NP ; We claim to resolve the P=?NP problem via a formal argument for P=NP.']

In [ ]:
len(lines), data.shape

(41000, (41000, 9))

### Tokenization

You know the dril. The data is messy. Go clean the data. Use WordPunctTokenizer or something.


In [4]:
# Task: convert lines (in-place) into strings of space-separated tokens. Import & use WordPunctTokenizer
from nltk.tokenize import WordPunctTokenizer

tokenizer = WordPunctTokenizer()
lines = [' '.join(seq).lower() for seq in [tokenizer.tokenize(i) for i in lines]]

In [ ]:
lines[0]

'dual recurrent attention units for visual question answering ; we propose an architecture for vqa which utilizes recurrent layers to generate visual and textual attention . the memory characteristic of the proposed recurrent attention units offers a rich joint embedding of visual and textual features and enables the model to reason relations between several parts of the image and question . our single model outperforms the first place winner on the vqa 1 . 0 dataset , performs within margin to the current state - of - the - art ensemble model . we also experiment with replacing attention mechanisms in other state - of - the - art models with our implementation and show increased accuracy . in both cases , our recurrent attention mechanism improves performance in tasks requiring sequential or relational reasoning on the vqa dataset .'

In [5]:
assert sorted(lines, key=len)[0] == \
    'differential contrastive divergence ; this paper has been retracted .'
assert sorted(lines, key=len)[2] == \
    'p = np ; we claim to resolve the p =? np problem via a formal argument for p = np .'

In [ ]:
len(lines)

41000

### N-Gram Language Model (1point)

A language model is a probabilistic model that estimates text probability: the joint probability of all tokens $w_t$ in text $X$: $P(X) = P(w_1, \dots, w_T)$.

It can do so by following the chain rule:
$$P(w_1, \dots, w_T) = P(w_1)P(w_2 \mid w_1)\dots P(w_T \mid w_1, \dots, w_{T-1}).$$

The problem with such approach is that the final term $P(w_T \mid w_1, \dots, w_{T-1})$ depends on $n-1$ previous words. This probability is impractical to estimate for long texts, e.g. $T = 1000$.

One popular approximation is to assume that next word only depends on a finite amount of previous words:

$$P(w_t \mid w_1, \dots, w_{t - 1}) = P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1})$$

Such model is called __n-gram language model__ where n is a parameter. For example, in 3-gram language model, each word only depends on 2 previous words.

$$
    P(w_1, \dots, w_n) = \prod_t P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1}).
$$

You can also sometimes see such approximation under the name of _n-th order markov assumption_.

The first stage to building such a model is counting all word occurences given N-1 previous words

In [6]:
from tqdm import tqdm
from collections import defaultdict, Counter

# special tokens:
# - `UNK` represents absent tokens, # PAD would be better, but it isn't here
# - `EOS` is a special token after the end of sequence

UNK, EOS = "_UNK_", "_EOS_"

def count_ngrams(lines, n):
    """
    Count how many times each word occured after (n - 1) previous words
    :param lines: an iterable of strings with space-separated tokens
    :returns: a dictionary { tuple(prefix_tokens): {next_token_1: count_1, next_token_2: count_2}}

    When building counts, please consider the following two edge cases:
    - if prefix is shorter than (n - 1) tokens, it should be padded with UNK. For n=3,
      empty prefix: "" -> (UNK, UNK)
      short prefix: "the" -> (UNK, the)
      long prefix: "the new approach" -> (new, approach)
    - you should add a special token, EOS, at the end of each sequence
      "... with deep neural networks ." -> (..., with, deep, neural, networks, ., EOS)
      count the probability of this token just like all others.
    """
    counts = defaultdict(Counter)
    # counts[(word1, word2)][word3] = how many times word3 occured after (word1, word2)

    for seq in lines:
        seq = seq.split()
        for word_idx in range(len(seq)):
            if word_idx == 0:
                counts[(UNK, UNK)][seq[word_idx]] += 1
            elif word_idx == 1:
                counts[(UNK, seq[word_idx-1])][seq[word_idx]] += 1
            else:
                counts[(seq[word_idx-2], seq[word_idx-1])][seq[word_idx]] += 1
        counts[(seq[len(seq)-2], seq[len(seq)-1])][EOS] += 1 # обработка конца последовательности

    return counts

In [ ]:
counts = count_ngrams(lines, 3)

In [ ]:
counts[('_UNK_', '_UNK_')].most_common(n=1), counts[('_UNK_', 'a')].most_common(n=1), counts[('a', 'new')].most_common(n=1)

([('a', 3207)], [('new', 167)], [('approach', 454)])

In [7]:
# let's test it
dummy_lines = sorted(lines, key=len)[:100]
dummy_counts = count_ngrams(dummy_lines, n=3)
assert set(map(len, dummy_counts.keys())) == {2}, "please only count {n-1}-grams"
assert len(dummy_counts[('_UNK_', '_UNK_')]) == 78
assert dummy_counts['_UNK_', 'a']['note'] == 3
assert dummy_counts['p', '=']['np'] == 2
assert dummy_counts['author', '.']['_EOS_'] == 1

Once we can count N-grams, we can build a probabilistic language model.
The simplest way to compute probabilities is in proporiton to counts:

$$ P(w_t | prefix) = { Count(prefix, w_t) \over \sum_{\hat w} Count(prefix, \hat w) } $$

In [ ]:
#counts = count_ngrams(lines, 3)
probabilities = defaultdict(Counter)
for prefix in counts.keys():
    for word in counts[prefix]:
        sum_of_all_words = sum([counts[prefix][i] for i in counts[prefix].keys()])
        count_this_word = counts[prefix][word]
        probabilities[prefix][word] = count_this_word / sum_of_all_words

In [ ]:
prefix = ('attention', 'units')
probabilities[prefix].most_common(n=3)

[('for', 0.5), ('offers', 0.5)]

In [ ]:
prefix = 'attention units'
prefix = prefix.split()
prefix = prefix[max(0, len(prefix) - 3 + 1):]
prefix = [ UNK ] * (3 - 1 - len(prefix)) + prefix
probabilities[tuple(prefix)]

Counter({'for': 0.5, 'offers': 0.5})

In [ ]:
for prefix in counts.keys():
    print(prefix)
    break

('_UNK_', '_UNK_')


In [8]:
class NGramLanguageModel:
    def __init__(self, lines, n):
        """
        Train a simple count-based language model:
        compute probabilities P(w_t | prefix) given ngram counts

        :param n: computes probability of next token given (n - 1) previous words
        :param lines: an iterable of strings with space-separated tokens
        """
        assert n >= 1
        self.n = n

        counts = count_ngrams(lines, self.n)

        # compute token proabilities given counts
        self.probs = defaultdict(Counter)
        # probs[(word1, word2)][word3] = P(word3 | word1, word2)

        # populate self.probs with actual probabilities
        for prefix in counts.keys():
            for word in counts[prefix]:
                sum_of_all_words = sum([counts[prefix][i] for i in counts[prefix].keys()])
                count_this_word = counts[prefix][word]
                self.probs[prefix][word] = count_this_word / sum_of_all_words

    def get_possible_next_tokens(self, prefix):
        """
        :param prefix: string with space-separated prefix tokens
        :returns: a dictionary {token : it's probability} for all tokens with positive probabilities
        """
        prefix = prefix.split()
        prefix = prefix[max(0, len(prefix) - self.n + 1):]
        prefix = [ UNK ] * (self.n - 1 - len(prefix)) + prefix
        return self.probs[tuple(prefix)]

    def get_next_token_prob(self, prefix, next_token):
        """
        :param prefix: string with space-separated prefix tokens
        :param next_token: the next token to predict probability for
        :returns: P(next_token|prefix) a single number, 0 <= P <= 1
        """
        min_prob = 1e-10
        prob = self.get_possible_next_tokens(prefix).get(next_token, min_prob)
        return max(prob, min_prob)

Let's test it!

In [9]:
dummy_lm = NGramLanguageModel(dummy_lines, n=3)

p_initial = dummy_lm.get_possible_next_tokens('') # '' -> ['_UNK_', '_UNK_']
assert np.allclose(p_initial['learning'], 0.02)
assert np.allclose(p_initial['a'], 0.13)
assert np.allclose(p_initial.get('meow', 0), 0)
assert np.allclose(sum(p_initial.values()), 1)

p_a = dummy_lm.get_possible_next_tokens('a') # '' -> ['_UNK_', 'a']
assert np.allclose(p_a['machine'], 0.15384615)
assert np.allclose(p_a['note'], 0.23076923)
assert np.allclose(p_a.get('the', 0), 0)
assert np.allclose(sum(p_a.values()), 1)

assert np.allclose(dummy_lm.get_possible_next_tokens('a note')['on'], 1)
assert dummy_lm.get_possible_next_tokens('a machine') == \
    dummy_lm.get_possible_next_tokens("there have always been ghosts in a machine"), \
    "your 3-gram model should only depend on 2 previous words"

Now that you've got a working n-gram language model, let's see what sequences it can generate. But first, let's train it on the whole dataset.

In [10]:
lm = NGramLanguageModel(lines, n=3)

In [ ]:
prefix = 'there have'
lm.get_next_token_prob(prefix, 'been')

0.9083969465648855

In [ ]:
lm.get_possible_next_tokens('hilbert')

Counter({'space': 0.8, "'": 0.2})

In [ ]:
prefix = 'it was necessary to'
print(end=prefix + " ", flush=True)

for i in range(100):
    tokens, probs = zip(*lm.get_possible_next_tokens(prefix).items())
    next_token = np.random.choice(tokens, p=probs)
    prefix += " " + next_token
    print(end=next_token + " ", flush=True)

it was necessary to take advantage of recent qe tasks despite not having a two - stage network . dcr is able to handle the problem of velocity trajectories , our model can beneficially be used for image - based classifier and quantum modeling '), and show that it learned about the geometry of scene understanding difficult . in visual - inertial sensor data to examine some key phenomena of homophily in independence movements in their favorite movie without changing the threshold of misspellings differences at the output distribution defined on small to large domains . therefore , networks trained to select actions . 

In [ ]:
tokens, probs = zip(*lm.get_possible_next_tokens('machine learning is').items())

next_token = np.random.choice(tokens, p=probs)
print(next_token)

designed


The process of generating sequences is... well, it's sequential. You maintain a list of tokens and iteratively add next token by sampling with probabilities.

$ X = [] $

__forever:__
* $w_{next} \sim P(w_{next} | X)$
* $X = concat(X, w_{next})$


Instead of sampling with probabilities, one can also try always taking most likely token, sampling among top-K most likely tokens or sampling with temperature. In the latter case (temperature), one samples from

$$w_{next} \sim {P(w_{next} | X) ^ {1 / \tau} \over \sum_{\hat w} P(\hat w | X) ^ {1 / \tau}}$$

Where $\tau > 0$ is model temperature. If $\tau << 1$, more likely tokens will be sampled with even higher probability while less likely tokens will vanish.

In [ ]:
tokens = lm.get_possible_next_tokens('hilbert').keys()
tokens

dict_keys(['space', "'"])

In [11]:
def get_next_token(lm, prefix, temperature=1.0):
    """
    return next token after prefix;
    :param temperature: samples proportionally to lm probabilities ^ (1 / temperature)
        if temperature == 0, always takes most likely token. Break ties arbitrarily.
    """
    prob = defaultdict()
    tokens = lm.get_possible_next_tokens(prefix).keys()

    whole_prob = 0
    if temperature == 0:
        maxprob = 0
        maxtoken = ''
        for word in tokens:
            current_word_prob = lm.get_next_token_prob(prefix, word)
            if maxprob <= current_word_prob:
                maxprob = current_word_prob
                maxtoken = word
        return maxtoken

    else:
        for word in tokens:
            current_word_prob = lm.get_next_token_prob(prefix, word)
            temp_prob = current_word_prob ** (1 / temperature)
            whole_prob += temp_prob
            prob[word] = temp_prob

        for word in tokens:
            prob[word] /= whole_prob

    tokens, probs = zip(*prob.items())
    next_token = np.random.choice(tokens, p=probs)
    return next_token

In [12]:
from collections import Counter
test_freqs = Counter([get_next_token(lm, 'there have') for _ in range(10000)])
assert 250 < test_freqs['not'] < 450
assert 8500 < test_freqs['been'] < 9500
assert 1 < test_freqs['lately'] < 200

test_freqs = Counter([get_next_token(lm, 'deep', temperature=1.0) for _ in range(10000)])
assert 1500 < test_freqs['learning'] < 3000
test_freqs = Counter([get_next_token(lm, 'deep', temperature=0.5) for _ in range(10000)])
assert 8000 < test_freqs['learning'] < 9000
test_freqs = Counter([get_next_token(lm, 'deep', temperature=0.0) for _ in range(10000)])
assert test_freqs['learning'] == 10000

print("Looks nice!")

Looks nice!


Let's have fun with this model

In [ ]:
prefix = 'artificial' # <- your ideas :)

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break

print(prefix)

artificial neural networks that use small - sample convergence rates can be regarded as a shared reconstruction graph ( radag ), strong elimination of connections between these functions in deep learning on distributed learning in vector space , but in recent years as an associative memory proposed by cognitive psychology and economics . traditional optical flow estimation . in this setting and $\ exp (- u ^{\ top } \ textrm { polylog }( n +\ sqrt { d / n ). in this paper , we can compare the performance of the standard tools in machine learning algorithms . we


In [ ]:
prefix = 'bridging the' # <- more of your ideas

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix, temperature=0.5)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break

print(prefix)

bridging the gap between the two - dimensional settings , we propose a novel approach to the best of our system achieves state - of - the - art performance on a wide range of applications . _EOS_


__More in the homework:__ nucleus sampling, top-k sampling, beam search(not for the faint of heart).

### Evaluating language models: perplexity (1point)

Perplexity is a measure of how well your model approximates the true probability distribution behind the data. __Smaller perplexity = better model__.

To compute perplexity on one sentence, use:
$$
    {\mathbb{P}}(w_1 \dots w_N) = P(w_1, \dots, w_N)^{-\frac1N} = \left( \prod_t P(w_t \mid w_{t - n}, \dots, w_{t - 1})\right)^{-\frac1N},
$$


On the corpora level, perplexity is a product of probabilities of all tokens in all sentences to the power of $1/N$, where $N$ is __total length (in tokens) of all sentences__ in corpora.

This number can quickly get too small for float32/float64 precision, so we recommend you to first compute log-perplexity (from log-probabilities) and then take the exponent.

In [75]:
def perplexity(lm, n, lines, min_logprob=np.log(10 ** -50.)):
    """
    :param lines: a list of strings with space-separated tokens
    :param min_logprob: if log(P(w | ...)) is smaller than min_logprop, set it equal to min_logrob
    :returns: corpora-level perplexity - a single scalar number from the formula above

    Note: do not forget to compute P(w_first | empty) and P(eos | full_sequence)

    PLEASE USE lm.get_next_token_prob and NOT lm.get_possible_next_tokens
    для лог-вероятностей такая формула:
        L = 1/N * Σ(-log(P(w_i)))
        Perplexity = exp(L)
    """
    all_log_probs_for_sum = []
    for seq in lines:
        proba_first = lm.get_next_token_prob("", seq.split()[0])
        proba_eos = lm.get_next_token_prob(seq, EOS)

        log_prob_first = -np.log(max(proba_first, np.exp(min_logprob)))
        log_prob_eos = -np.log(max(proba_eos, np.exp(min_logprob)))

        all_log_probs_for_sum.append(log_prob_first)
        all_log_probs_for_sum.append(log_prob_eos)

        context = [seq.split()[0]]
        seq = seq.split()[1:]
        for word in seq:
            proba = lm.get_next_token_prob(' '.join(context), word)
            log_prob = -np.log(max(proba, np.exp(min_logprob)))

            all_log_probs_for_sum.append(log_prob)
            if len(context) < (n-1):
                context.append(word)
            elif len(context) == (n-1):
                context = context[1:]
                context.append(word)

    N = len(all_log_probs_for_sum)
    L = (1 / N) * np.sum(all_log_probs_for_sum)
    Perplexity = np.exp(L)
    return Perplexity

In [ ]:
lm1 = NGramLanguageModel(dummy_lines, n=1)
lm3 = NGramLanguageModel(dummy_lines, n=3)
lm10 = NGramLanguageModel(dummy_lines, n=10)

ppx1 = perplexity(lm1, 1, dummy_lines)
ppx3 = perplexity(lm3, 3, dummy_lines)
ppx10 = perplexity(lm10, 10, dummy_lines)
ppx_missing = perplexity(lm3, 3, ['the jabberwock , with eyes of flame , '])  # thanks, L. Carrol

print("Perplexities: ppx1=%.3f ppx3=%.3f ppx10=%.3f" % (ppx1, ppx3, ppx10))

assert all(0 < ppx < 500 for ppx in (ppx1, ppx3, ppx10)), "perplexity should be non-negative and reasonably small"
assert ppx1 > ppx3 > ppx10, "higher N models should overfit and "
assert np.isfinite(ppx_missing) and ppx_missing > 10 ** 6, "missing words should have large but finite perplexity. " \
    " Make sure you use min_logprob right"
assert np.allclose([ppx1, ppx3, ppx10], (318.2132342216302, 1.5199996213739575, 1.1838145037901249))

Perplexities: ppx1=10000000000.000 ppx3=1.520 ppx10=10000000000.000


AssertionError: perplexity should be non-negative and reasonably small

Now let's measure the actual perplexity: we'll split the data into train and test and score model on test data only.

In [72]:
from sklearn.model_selection import train_test_split
train_lines, test_lines = train_test_split(lines, test_size=0.25, random_state=42)

In [ ]:
for n in (1, 2, 3):
    lm = NGramLanguageModel(n=n, lines=train_lines)
    ppx = perplexity(lm, n, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))

In [ ]:
# whoops, it just blew up :)

### LM Smoothing

The problem with our simple language model is that whenever it encounters an n-gram it has never seen before, it assigns it with the probabilitiy of 0. Every time this happens, perplexity explodes.

To battle this issue, there's a technique called __smoothing__. The core idea is to modify counts in a way that prevents probabilities from getting too low. The simplest algorithm here is Additive smoothing (aka [Lapace smoothing](https://en.wikipedia.org/wiki/Additive_smoothing)):

$$ P(w_t | prefix) = { Count(prefix, w_t) + \delta \over \sum_{\hat w} (Count(prefix, \hat w) + \delta) } $$

If counts for a given prefix are low, additive smoothing will adjust probabilities to a more uniform distribution. Not that the summation in the denominator goes over _all words in the vocabulary_.

Here's an example code we've implemented for you:

In [13]:
class LaplaceLanguageModel(NGramLanguageModel):
    """ this code is an example, no need to change anything """
    def __init__(self, lines, n, delta=1.0):
        self.n = n
        counts = count_ngrams(lines, self.n)
        self.vocab = set(token for token_counts in counts.values() for token in token_counts)
        self.probs = defaultdict(Counter)

        for prefix in counts:
            token_counts = counts[prefix]
            total_count = sum(token_counts.values()) + delta * len(self.vocab)
            self.probs[prefix] = {token: (token_counts[token] + delta) / total_count
                                          for token in token_counts}
    def get_possible_next_tokens(self, prefix):
        token_probs = super().get_possible_next_tokens(prefix)
        missing_prob_total = 1.0 - sum(token_probs.values())
        missing_prob = missing_prob_total / max(1, len(self.vocab) - len(token_probs))
        return {token: token_probs.get(token, missing_prob) for token in self.vocab}

    def get_next_token_prob(self, prefix, next_token):
        token_probs = super().get_possible_next_tokens(prefix)
        if next_token in token_probs:
            return token_probs[next_token]
        else:
            missing_prob_total = 1.0 - sum(token_probs.values())
            missing_prob_total = max(0, missing_prob_total) # prevent rounding errors
            return missing_prob_total / max(1, len(self.vocab) - len(token_probs))


**Disclaimer**: the implementation above assumes all words unknown within a given context to be equally likely, *as well as the words outside of vocabulary*. Therefore, its' perplexity will be lower than it should when encountering such words. Therefore, comparing it with a model with fewer unknown words will not be fair. When implementing your own smoothing, you may handle this by adding a virtual `UNK` token of non-zero probability. Technically, this will result in a model where probabilities do not add up to $1$, but it is close enough for a practice excercise.

In [14]:
#test that it's a valid probability model
for n in (1, 2, 3):
    dummy_lm = LaplaceLanguageModel(dummy_lines, n=n)
    assert np.allclose(sum([dummy_lm.get_next_token_prob('a', w_i) for w_i in dummy_lm.vocab]), 1), "I told you not to break anything! :)"

In [ ]:
for n in (1, 2, 3):
    lm = LaplaceLanguageModel(train_lines, n=n, delta=0.1)
    ppx = perplexity(lm, n, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))

N = 1, Perplexity = 54176.00000
N = 2, Perplexity = 54176.00000
N = 3, Perplexity = 3679.44765


In [ ]:
# optional: try to sample tokens from such a model

### Kneser-Ney smoothing (2 points)


Additive smoothing is simple, reasonably good but definitely not a State of The Art algorithm.


Your final task in this notebook is to implement [Kneser-Ney](https://en.wikipedia.org/wiki/Kneser%E2%80%93Ney_smoothing) smoothing.

It can be computed recurrently, for n>1:

$$P_{kn}(w_t | prefix_{n-1}) = { \max(0, Count(prefix_{n-1}, w_t) - \delta) \over \sum_{\hat w} Count(prefix_{n-1}, \hat w)} + \lambda_{prefix_{n-1}} \cdot P_{kn}(w_t | prefix_{n-2})$$

where
- $prefix_{n-1}$ is a tuple of {n-1} previous tokens
- $lambda_{prefix_{n-1}}$ is a normalization constant chosen so that probabilities add up to 1
- Unigram $P_{kn}(w_t | prefix_{n-2})$ corresponds to Kneser Ney smoothing for {N-1}-gram language model.
- Unigram $P_{kn}(w_t)$ is a special case: how likely it is to see x_t in an unfamiliar context

See lecture slides or wiki for more detailed formulae.

__Your task__ is to
- implement `KneserNeyLanguageModel` class,
- test it on 1-3 gram language models
- find optimal (within reason) smoothing delta for 3-gram language model with Kneser-Ney smoothing

In [19]:
from nltk.util import ngrams

counter2 = Counter()
k = 0
for line in lines:
    if k > 3:
        break
    ngram_counts = Counter(ngrams(line.split(), 3))
    for key, value in ngram_counts.items():
        counter2[key] += value
    k += 1

In [65]:
from collections import defaultdict, Counter
from nltk.util import ngrams

class KneserNeyLanguageModel(NGramLanguageModel):
    """ A template for Kneser-Ney language model. Default delta may be suboptimal. """
    def __init__(self, lines, n, delta=0.75):
        self.n = n
        self.delta = delta
        self.ngram_counts = defaultdict(Counter)
        self.context_counts = defaultdict(Counter)
        self.continuation_counts = Counter()
        self.vocab = set()

        for line in lines:
            tokens = line.split()
            for n in range(1, self.n+1):
                for ngram in ngrams(tokens, n):
                    self.vocab.update(ngram)
                    if n == 1:
                        self.continuation_counts[ngram[-1]] += 1
                    else:
                        self.ngram_counts[ngram[:-1]][ngram[-1]] += 1
                        self.context_counts[ngram[:-1]].update([ngram[-1]])

    def get_possible_next_tokens(self, prefix):
        return list(self.ngram_counts[prefix].keys())

    def get_next_token_prob(self, prefix, next_token):
        if not prefix:
            prob = max(self.continuation_counts[next_token] - self.delta, 0) / sum(self.continuation_counts.values()) + (self.delta / len(self.vocab)) * (len(self.vocab) / len(self.vocab))

        elif prefix not in self.context_counts or not self.context_counts[prefix]:
            prob = self.get_next_token_prob(' '.join(prefix[1:]), next_token) if len(prefix) > 1 else self.continuation_counts[next_token] / sum(self.continuation_counts.values())

        else:
            adjusted_count = max(self.ngram_counts[prefix][next_token] - self.delta, 0)
            prefix_sum = sum(self.ngram_counts[prefix].values())
            lambda_value = self.delta * len(self.ngram_counts[prefix]) / prefix_sum
            continuation_prob = self.get_next_token_prob(' '.join(prefix[1:]), next_token) if len(prefix) > 1 else self.continuation_counts[next_token] / sum(self.continuation_counts.values())
            prob = adjusted_count / prefix_sum + lambda_value * continuation_prob

        return prob

In [66]:
dummy_lm = KneserNeyLanguageModel(dummy_lines, n=1)

In [67]:
sum([dummy_lm.get_next_token_prob('a', w_i) for w_i in dummy_lm.vocab])

0.9999999999999954

In [68]:
#test that it's a valid probability model
for n in (1, 2, 3):
    dummy_lm = KneserNeyLanguageModel(dummy_lines, n=n)
    assert np.allclose(sum([dummy_lm.get_next_token_prob('a', w_i) for w_i in dummy_lm.vocab]), 1), "I told you not to break anything! :)"

In [ ]:
# не хватило ОЗУ просто
for n in (1, 2, 3):
    lm = KneserNeyLanguageModel(train_lines, n=n, delta=0.75)
    ppx = perplexity(lm, n=n, lines=test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))